# DISCLAIMER AND LIABILITY

This website provides clinical test data for informational purposes and the convenience of the public. CDISC does not control or guarantee the currency, accuracy, relevance, or completeness of the data. The data has been analyzed, cleansed, and aggregated where appropriate to facilitate use and discussion in research. By downloading / using this data, you agree to the Terms of use.

## Terms of Use

You shall not (and will not allow or assist any third party to), under any circumstances: (i) mislead, confuse, or cause misapprehension or confusion among users of the data as to the features, functionality, origin, capabilities, or other aspects of the data, (b) disassemble, reverse engineer, decompile, modify, or alter any part of the Data; (c) use the Data in any manner or for any purpose that violates any law or regulation, (d) use the Data in order to compete with CDISC, (e) sublicense or distribute the data for a fee, or (f) sublicense or distribute the data without an attribution back to CDISC.

UNLESS REQUIRED BY APPLICABLE LAW, ACCESS AND USE OF THE DATA IS PROVIDED BY CDISC AND ITS CONSTITUENT PARTS (INCLUDING, BUT NOT LIMITED TO THE CDISC BOARD OF DIRECTORS, CDISC EMPLOYEES, AND CDISC MEMBERS, PARTICIPANTS, CONTRACTORS, AND REPRESENTATIVES) "AS IS" AND WITHOUT ANY WARRANTIES WHATSOEVER, WHETHER EXPRESS, IMPLIED, STATUTORY, OR OTHERWISE, AND CDISC AND ITS CONSTITUENT PARTS (INCLUDING, BUT NOT LIMITED TO THE CDISC BOARD OF DIRECTORS, CDISC EMPLOYEES, AND CDISC MEMBERS, PARTICIPANTS, CONTRACTORS, AND REPRESENTATIVES) EXPRESSLY DISCLAIM ANY WARRANTY OF  MERCHANTABILITY, TITLE, NONINFRINGEMENT, FITNESS FOR A PARTICULAR OR INTENDED PURPOSE, OR ANY OTHER WARRANTY OTHERWISE ARISING OUT OF THIS LETTER AGREEMENT, INCLUDING ACCESS OR USE OF THE DATA. You are solely responsible for determining the appropriateness of accessing and/or using the data and assume any risks associated with your access and/or use.

IN NO EVENT AND UNDER NO LEGAL THEORY, WHETHER IN TORT (INCLUDING NEGLIGENCE), CONTRACT, OR OTHERWISE, UNLESS REQUIRED BY APPLICABLE LAW (SUCH AS DELIBERATE AND GROSSLY NEGLIGENT ACTS) OR AGREED TO IN WRITING, SHALL CDISC, ANY OF CDISC’S CONSTITUENT PARTS (INCLUDING, BUT NOT LIMITED TO THE CDISC BOARD OF DIRECTORS, THE CDISC EMPLOYEES, OR CDISC MEMBERS, PARTICIPANTS, CONTRACTORS, OR REPRESENTATIVES) BE LIABLE FOR DAMAGES, INCLUDING ANY DIRECT, INDIRECT, SPECIAL, EXEMPLARY, INCIDENTAL, OR CONSEQUENTIAL DAMAGES OF ANY CHARACTER ARISING IN ANY WAY AS A RESULT OF THIS LETTER AGREEMENT OR OUT OF THE USE OR INABILITY TO USE THE DATA (INCLUDING DAMAGES FOR LOSS OF GOODWILL, LOSS OF PROFITS, LOSS OF USE, OR BUSINESS INTERRUPTION), EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGES.  THIS LIMITATION OF DAMAGES AND CLAIMS IS INTENDED TO APPLY TO ALL CLAIMS WITHOUT REGARD TO WHICH OTHER PROVISIONS OF THIS LETTER AGREEMENT HAVE BEEN BREACHED OR PROVEN INEFFECTIVE.

---

# CDISC Oncology ETL: Transformations for DM, AE, and LB Datasets

### Imports

In [2]:
import pandas as pd
import pyreadstat
import os

# DM.xpt data

### Load raw DM.xpt

In [3]:
def load_dm_data():
    data_path = r'updated-pilot-submission-package\900172\m5\datasets\cdiscpilot01\tabulations\sdtm'
    dm_df, dm_meta = pyreadstat.read_xport(os.path.join(data_path, 'dm.xpt'))
    return dm_df

dm_df = load_dm_data()
dm_df.head(5)

dm_df.columns

Index(['STUDYID', 'DOMAIN', 'USUBJID', 'SUBJID', 'RFSTDTC', 'RFENDTC',
       'RFXSTDTC', 'RFXENDTC', 'RFICDTC', 'RFPENDTC', 'DTHDTC', 'DTHFL',
       'SITEID', 'AGE', 'AGEU', 'SEX', 'RACE', 'ETHNIC', 'ARMCD', 'ARM',
       'ACTARMCD', 'ACTARM', 'COUNTRY', 'DMDTC', 'DMDY'],
      dtype='object')

### 1. Missing Value Imputation

In [3]:
def impute_missing(df):
    df = df.copy()
    df['RACE'] = df['RACE'].fillna('Unknown')
    df['ETHNIC'] = df['ETHNIC'].fillna('Unknown')
    df['COUNTRY'] = df['COUNTRY'].fillna(df['COUNTRY'].mode()[0] if not df['COUNTRY'].mode().empty else 'Unknown')
    df['DMDY'] = df['DMDY'].fillna(df['DMDY'].median())
    df.dropna(subset=['USUBJID'], inplace=True)
    return df

impute_missing_df = impute_missing(dm_df)
impute_missing_df.head(5)
    

,STUDYID,DOMAIN,USUBJID,SUBJID,RFSTDTC,RFENDTC,RFXSTDTC,RFXENDTC,RFICDTC,RFPENDTC,...,SEX,RACE,ETHNIC,ARMCD,ARM,ACTARMCD,ACTARM,COUNTRY,DMDTC,DMDY
0,CDISCPILOT01,DM,01-701-1015,1015,2014-01-02,2014-07-02,2014-01-02,2014-07-02,,2014-07-02T11:45,...,F,WHITE,HISPANIC OR LATINO,Pbo,Placebo,Pbo,Placebo,USA,2013-12-26,-7.0
1,CDISCPILOT01,DM,01-701-1023,1023,2012-08-05,2012-09-02,2012-08-05,2012-09-01,,2013-02-18,...,M,WHITE,HISPANIC OR LATINO,Pbo,Placebo,Pbo,Placebo,USA,2012-07-22,-14.0
2,CDISCPILOT01,DM,01-701-1028,1028,2013-07-19,2014-01-14,2013-07-19,2014-01-14,,2014-01-14T11:10,...,M,WHITE,NOT HISPANIC OR LATINO,Xan_Hi,Xanomeline High Dose,Xan_Hi,Xanomeline High Dose,USA,2013-07-11,-8.0
3,CDISCPILOT01,DM,01-701-1033,1033,2014-03-18,2014-04-14,2014-03-18,2014-03-31,,2014-09-15,...,M,WHITE,NOT HISPANIC OR LATINO,Xan_Lo,Xanomeline Low Dose,Xan_Lo,Xanomeline Low Dose,USA,2014-03-10,-8.0
4,CDISCPILOT01,DM,01-701-1034,1034,2014-07-01,2014-12-30,2014-07-01,2014-12-30,,2014-12-30T09:50,...,F,WHITE,NOT HISPANIC OR LATINO,Xan_Hi,Xanomeline High Dose,Xan_Hi,Xanomeline High Dose,USA,2014-06-24,-7.0


### 2. Date Standardization

In [4]:
def standardize_dates(df):
    df = df.copy()
    date_cols = ['RFSTDTC', 'RFENDTC', 'RFXSTDTC', 'RFXENDTC', 'RFICDTC', 'RFPENDTC', 'DTHDTC', 'DMDTC']
    for col in date_cols:
        if col in dm_df.columns:
           df[col] = pd.to_datetime(dm_df[col], errors='coerce')
    return df

standardize_df = standardize_dates(impute_missing_df)
standardize_df.head(5)

,STUDYID,DOMAIN,USUBJID,SUBJID,RFSTDTC,RFENDTC,RFXSTDTC,RFXENDTC,RFICDTC,RFPENDTC,...,SEX,RACE,ETHNIC,ARMCD,ARM,ACTARMCD,ACTARM,COUNTRY,DMDTC,DMDY
0,CDISCPILOT01,DM,01-701-1015,1015,2014-01-02,2014-07-02,2014-01-02,2014-07-02,NaT,2014-07-02 11:45:00,...,F,WHITE,HISPANIC OR LATINO,Pbo,Placebo,Pbo,Placebo,USA,2013-12-26,-7.0
1,CDISCPILOT01,DM,01-701-1023,1023,2012-08-05,2012-09-02,2012-08-05,2012-09-01,NaT,NaT,...,M,WHITE,HISPANIC OR LATINO,Pbo,Placebo,Pbo,Placebo,USA,2012-07-22,-14.0
2,CDISCPILOT01,DM,01-701-1028,1028,2013-07-19,2014-01-14,2013-07-19,2014-01-14,NaT,2014-01-14 11:10:00,...,M,WHITE,NOT HISPANIC OR LATINO,Xan_Hi,Xanomeline High Dose,Xan_Hi,Xanomeline High Dose,USA,2013-07-11,-8.0
3,CDISCPILOT01,DM,01-701-1033,1033,2014-03-18,2014-04-14,2014-03-18,2014-03-31,NaT,NaT,...,M,WHITE,NOT HISPANIC OR LATINO,Xan_Lo,Xanomeline Low Dose,Xan_Lo,Xanomeline Low Dose,USA,2014-03-10,-8.0
4,CDISCPILOT01,DM,01-701-1034,1034,2014-07-01,2014-12-30,2014-07-01,2014-12-30,NaT,2014-12-30 09:50:00,...,F,WHITE,NOT HISPANIC OR LATINO,Xan_Hi,Xanomeline High Dose,Xan_Hi,Xanomeline High Dose,USA,2014-06-24,-7.0


### 3. Data Type Enforcement (per CDISC specs)

In [5]:
def enforce_data_types(df):
    df = df.copy()
    df['AGE'] = pd.to_numeric(df['AGE'], errors='coerce').astype('Int64')
    df['AGEU'] = df['AGEU'].astype('category')
    df['SEX'] = df['SEX'].astype('category')
    df['RACE'] = df['RACE'].astype('category')
    df['ETHNIC'] = df['ETHNIC'].astype('category')
    df['STUDYID'] = df['STUDYID'].astype(str)
    df['DMDY'] = pd.to_numeric(df['DMDY'], errors='coerce').astype('Int64')
    df['DTHFL'] = df['DTHFL'].astype('category')
    return df

enforce_data_df = enforce_data_types(standardize_df)
enforce_data_df.head(5)

,STUDYID,DOMAIN,USUBJID,SUBJID,RFSTDTC,RFENDTC,RFXSTDTC,RFXENDTC,RFICDTC,RFPENDTC,...,SEX,RACE,ETHNIC,ARMCD,ARM,ACTARMCD,ACTARM,COUNTRY,DMDTC,DMDY
0,CDISCPILOT01,DM,01-701-1015,1015,2014-01-02,2014-07-02,2014-01-02,2014-07-02,NaT,2014-07-02 11:45:00,...,F,WHITE,HISPANIC OR LATINO,Pbo,Placebo,Pbo,Placebo,USA,2013-12-26,-7
1,CDISCPILOT01,DM,01-701-1023,1023,2012-08-05,2012-09-02,2012-08-05,2012-09-01,NaT,NaT,...,M,WHITE,HISPANIC OR LATINO,Pbo,Placebo,Pbo,Placebo,USA,2012-07-22,-14
2,CDISCPILOT01,DM,01-701-1028,1028,2013-07-19,2014-01-14,2013-07-19,2014-01-14,NaT,2014-01-14 11:10:00,...,M,WHITE,NOT HISPANIC OR LATINO,Xan_Hi,Xanomeline High Dose,Xan_Hi,Xanomeline High Dose,USA,2013-07-11,-8
3,CDISCPILOT01,DM,01-701-1033,1033,2014-03-18,2014-04-14,2014-03-18,2014-03-31,NaT,NaT,...,M,WHITE,NOT HISPANIC OR LATINO,Xan_Lo,Xanomeline Low Dose,Xan_Lo,Xanomeline Low Dose,USA,2014-03-10,-8
4,CDISCPILOT01,DM,01-701-1034,1034,2014-07-01,2014-12-30,2014-07-01,2014-12-30,NaT,2014-12-30 09:50:00,...,F,WHITE,NOT HISPANIC OR LATINO,Xan_Hi,Xanomeline High Dose,Xan_Hi,Xanomeline High Dose,USA,2014-06-24,-7


### 4. Derived Variables (insights-focused for oncology)

In [6]:
def derive_variables(df):
    df = df.copy()
    
    def age_group(age):
        if pd.isna(age):
            return 'Unknown'
        elif age < 18:
            return 'Pediatric'
        elif age <= 65:
            return 'Adult'
        else:
            return 'Senior'
    df['AGE_GROUP'] = df['AGE'].apply(age_group)
    
    df['STUDY_DURATION'] = df['DMDY'].fillna(0)
    if 'RFXENDTC' in df.columns and 'RFXSTDTC' in df.columns:
        df['STUDY_DURATION'] = df['STUDY_DURATION'].fillna((df['RFXENDTC'] - df['RFXSTDTC']).dt.days)
    
    df['ARM_TYPE'] = df['ARM'].str.lower().str.contains('placebo|control', na=False).map({True: 'Control', False: 'Treatment'}).fillna('Unknown')
    return df

derive_variables_df = derive_variables(enforce_data_df)
derive_variables_df.head(5)

,STUDYID,DOMAIN,USUBJID,SUBJID,RFSTDTC,RFENDTC,RFXSTDTC,RFXENDTC,RFICDTC,RFPENDTC,...,ARMCD,ARM,ACTARMCD,ACTARM,COUNTRY,DMDTC,DMDY,AGE_GROUP,STUDY_DURATION,ARM_TYPE
0,CDISCPILOT01,DM,01-701-1015,1015,2014-01-02,2014-07-02,2014-01-02,2014-07-02,NaT,2014-07-02 11:45:00,...,Pbo,Placebo,Pbo,Placebo,USA,2013-12-26,-7,Adult,-7,Control
1,CDISCPILOT01,DM,01-701-1023,1023,2012-08-05,2012-09-02,2012-08-05,2012-09-01,NaT,NaT,...,Pbo,Placebo,Pbo,Placebo,USA,2012-07-22,-14,Adult,-14,Control
2,CDISCPILOT01,DM,01-701-1028,1028,2013-07-19,2014-01-14,2013-07-19,2014-01-14,NaT,2014-01-14 11:10:00,...,Xan_Hi,Xanomeline High Dose,Xan_Hi,Xanomeline High Dose,USA,2013-07-11,-8,Senior,-8,Treatment
3,CDISCPILOT01,DM,01-701-1033,1033,2014-03-18,2014-04-14,2014-03-18,2014-03-31,NaT,NaT,...,Xan_Lo,Xanomeline Low Dose,Xan_Lo,Xanomeline Low Dose,USA,2014-03-10,-8,Senior,-8,Treatment
4,CDISCPILOT01,DM,01-701-1034,1034,2014-07-01,2014-12-30,2014-07-01,2014-12-30,NaT,2014-12-30 09:50:00,...,Xan_Hi,Xanomeline High Dose,Xan_Hi,Xanomeline High Dose,USA,2014-06-24,-7,Senior,-7,Treatment


### 5. Outlier Detection/Cleaning

In [11]:
def clean_outliers(df):
    df = df.copy()
    print("Before filters:", df.shape)
    
    # AGE: Keep if not NaN and in range
    df = df[(df['AGE'].notna()) & (df['AGE'] >= 0) & (df['AGE'] <= 120)]
    print("After AGE filter:", df.shape)
    
    df = df.drop_duplicates(subset=['USUBJID'])
    print("After duplicates:", df.shape)
    return df
    
clean_outliers_df = clean_outliers(derive_variables_df)
clean_outliers_df.head(5)

Before filters: (306, 28)
After AGE filter: (306, 28)
After duplicates: (306, 28)


,STUDYID,DOMAIN,USUBJID,SUBJID,RFSTDTC,RFENDTC,RFXSTDTC,RFXENDTC,RFICDTC,RFPENDTC,...,ARMCD,ARM,ACTARMCD,ACTARM,COUNTRY,DMDTC,DMDY,AGE_GROUP,STUDY_DURATION,ARM_TYPE
0,CDISCPILOT01,DM,01-701-1015,1015,2014-01-02,2014-07-02,2014-01-02,2014-07-02,NaT,2014-07-02 11:45:00,...,Pbo,Placebo,Pbo,Placebo,USA,2013-12-26,-7,Adult,-7,Control
1,CDISCPILOT01,DM,01-701-1023,1023,2012-08-05,2012-09-02,2012-08-05,2012-09-01,NaT,NaT,...,Pbo,Placebo,Pbo,Placebo,USA,2012-07-22,-14,Adult,-14,Control
2,CDISCPILOT01,DM,01-701-1028,1028,2013-07-19,2014-01-14,2013-07-19,2014-01-14,NaT,2014-01-14 11:10:00,...,Xan_Hi,Xanomeline High Dose,Xan_Hi,Xanomeline High Dose,USA,2013-07-11,-8,Senior,-8,Treatment
3,CDISCPILOT01,DM,01-701-1033,1033,2014-03-18,2014-04-14,2014-03-18,2014-03-31,NaT,NaT,...,Xan_Lo,Xanomeline Low Dose,Xan_Lo,Xanomeline Low Dose,USA,2014-03-10,-8,Senior,-8,Treatment
4,CDISCPILOT01,DM,01-701-1034,1034,2014-07-01,2014-12-30,2014-07-01,2014-12-30,NaT,2014-12-30 09:50:00,...,Xan_Hi,Xanomeline High Dose,Xan_Hi,Xanomeline High Dose,USA,2014-06-24,-7,Senior,-7,Treatment


### 6. Consistency Checks (CDISC controlled terms)

In [13]:
def consistency_checks(df):
    df = df.copy()
    df['SEX'] = df['SEX'].where(df['SEX'].isin(['M', 'F', 'U']), 'U')
    df['RACE'] = df['RACE'].astype(str)
    race_map = {
        'AMERICAN INDIAN OR ALASKA NATIVE': 'AMERICAN INDIAN/ALASKA NATIVE',
        'ASIAN': 'ASIAN',
        'BLACK OR AFRICAN AMERICAN': 'BLACK/AFRICAN AMERICAN',
        'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'NATIVE HAWAIIAN/OTHER PACIFIC ISLANDER',
        'WHITE': 'WHITE'
    }
    df['RACE'] = df['RACE'].replace(race_map)
    df['RACE'] = df['RACE'].astype('category')
    
    df['HIGH_RISK'] = ((df['AGE'] > 70) | 
                       (df['RACE'].str.contains('ASIAN|BLACK', na=False)) | 
                       (df['ARM_TYPE'] == 'Treatment')).astype(int)
    
    drop_cols = ['DOMAIN', 'SUBJID']
    df = df.drop(columns=[col for col in drop_cols if col in df.columns])
    return df
    
consistency_df = consistency_checks(clean_outliers_df)
consistency_df.head(5)

,STUDYID,USUBJID,RFSTDTC,RFENDTC,RFXSTDTC,RFXENDTC,RFICDTC,RFPENDTC,DTHDTC,DTHFL,...,ARM,ACTARMCD,ACTARM,COUNTRY,DMDTC,DMDY,AGE_GROUP,STUDY_DURATION,ARM_TYPE,HIGH_RISK
0,CDISCPILOT01,01-701-1015,2014-01-02,2014-07-02,2014-01-02,2014-07-02,NaT,2014-07-02 11:45:00,NaT,,...,Placebo,Pbo,Placebo,USA,2013-12-26,-7,Adult,-7,Control,0
1,CDISCPILOT01,01-701-1023,2012-08-05,2012-09-02,2012-08-05,2012-09-01,NaT,NaT,NaT,,...,Placebo,Pbo,Placebo,USA,2012-07-22,-14,Adult,-14,Control,0
2,CDISCPILOT01,01-701-1028,2013-07-19,2014-01-14,2013-07-19,2014-01-14,NaT,2014-01-14 11:10:00,NaT,,...,Xanomeline High Dose,Xan_Hi,Xanomeline High Dose,USA,2013-07-11,-8,Senior,-8,Treatment,1
3,CDISCPILOT01,01-701-1033,2014-03-18,2014-04-14,2014-03-18,2014-03-31,NaT,NaT,NaT,,...,Xanomeline Low Dose,Xan_Lo,Xanomeline Low Dose,USA,2014-03-10,-8,Senior,-8,Treatment,1
4,CDISCPILOT01,01-701-1034,2014-07-01,2014-12-30,2014-07-01,2014-12-30,NaT,2014-12-30 09:50:00,NaT,,...,Xanomeline High Dose,Xan_Hi,Xanomeline High Dose,USA,2014-06-24,-7,Senior,-7,Treatment,1


### 7. Export Prep (reorder for DB, add audit)

In [16]:
def prepare_export(df):
    """Step 7: Export Prep (reindex and timestamp)"""
    df = df.copy()
    col_order = [
        'USUBJID', 'STUDYID', 'SITEID', 'AGE', 'AGEU', 'SEX', 'RACE', 'ETHNIC', 'COUNTRY',
        'ARMCD', 'ARM', 'ACTARMCD', 'ACTARM', 'RFXSTDTC', 'RFXENDTC', 'DMDY', 'DMDTC',
        'AGE_GROUP', 'ARM_TYPE', 'STUDY_DURATION', 'HIGH_RISK'
    ]
    existing_cols = [col for col in col_order if col in df.columns]
    remaining_cols = [col for col in df.columns if col not in col_order]
    df = df.reindex(columns=existing_cols + remaining_cols)
    df['ETL_TIMESTAMP'] = pd.Timestamp.now()
    return df
    
prepare_export_df = prepare_export(consistency_df)
final_df = prepare_export_df
final_df.head(5)

,USUBJID,STUDYID,SITEID,AGE,AGEU,SEX,RACE,ETHNIC,COUNTRY,ARMCD,...,ARM_TYPE,STUDY_DURATION,HIGH_RISK,RFSTDTC,RFENDTC,RFICDTC,RFPENDTC,DTHDTC,DTHFL,ETL_TIMESTAMP
0,01-701-1015,CDISCPILOT01,701,63,YEARS,F,WHITE,HISPANIC OR LATINO,USA,Pbo,...,Control,-7,0,2014-01-02,2014-07-02,NaT,2014-07-02 11:45:00,NaT,,2025-09-24 19:51:49.658021
1,01-701-1023,CDISCPILOT01,701,64,YEARS,M,WHITE,HISPANIC OR LATINO,USA,Pbo,...,Control,-14,0,2012-08-05,2012-09-02,NaT,NaT,NaT,,2025-09-24 19:51:49.658021
2,01-701-1028,CDISCPILOT01,701,71,YEARS,M,WHITE,NOT HISPANIC OR LATINO,USA,Xan_Hi,...,Treatment,-8,1,2013-07-19,2014-01-14,NaT,2014-01-14 11:10:00,NaT,,2025-09-24 19:51:49.658021
3,01-701-1033,CDISCPILOT01,701,74,YEARS,M,WHITE,NOT HISPANIC OR LATINO,USA,Xan_Lo,...,Treatment,-8,1,2014-03-18,2014-04-14,NaT,NaT,NaT,,2025-09-24 19:51:49.658021
4,01-701-1034,CDISCPILOT01,701,77,YEARS,F,WHITE,NOT HISPANIC OR LATINO,USA,Xan_Hi,...,Treatment,-7,1,2014-07-01,2014-12-30,NaT,2014-12-30 09:50:00,NaT,,2025-09-24 19:51:49.658021


---

# AE.xpt Data

### Load raw AE.xpt

In [32]:
def load_ae_data():
    data_path = r'updated-pilot-submission-package\900172\m5\datasets\cdiscpilot01\tabulations\sdtm'
    ae_df, ae_meta = pyreadstat.read_xport(os.path.join(data_path, 'ae.xpt'))
    return ae_df

ae_df = load_ae_data()
ae_df.head(5)

,STUDYID,DOMAIN,USUBJID,AESEQ,AESPID,AETERM,AELLT,AELLTCD,AEDECOD,AEPTCD,...,AESDISAB,AESDTH,AESHOSP,AESLIFE,AESOD,AEDTC,AESTDTC,AEENDTC,AESTDY,AEENDY
0,CDISCPILOT01,AE,01-701-1015,1.0,E07,APPLICATION SITE ERYTHEMA,APPLICATION SITE REDNESS,NaN,APPLICATION SITE ERYTHEMA,NaN,...,N,N,N,N,N,2014-01-16,2014-01-03,,2.0,NaN
1,CDISCPILOT01,AE,01-701-1015,2.0,E08,APPLICATION SITE PRURITUS,APPLICATION SITE ITCHING,NaN,APPLICATION SITE PRURITUS,NaN,...,N,N,N,N,N,2014-01-16,2014-01-03,,2.0,NaN
2,CDISCPILOT01,AE,01-701-1015,3.0,E06,DIARRHOEA,DIARRHEA,NaN,DIARRHOEA,NaN,...,N,N,N,N,N,2014-01-16,2014-01-09,2014-01-11,8.0,10.0
3,CDISCPILOT01,AE,01-701-1023,3.0,E10,ATRIOVENTRICULAR BLOCK SECOND DEGREE,AV BLOCK SECOND DEGREE,NaN,ATRIOVENTRICULAR BLOCK SECOND DEGREE,NaN,...,N,N,N,N,N,2012-08-27,2012-08-26,,22.0,NaN
4,CDISCPILOT01,AE,01-701-1023,1.0,E08,ERYTHEMA,ERYTHEMA,NaN,ERYTHEMA,NaN,...,N,N,N,N,N,2012-08-27,2012-08-07,2012-08-30,3.0,26.0


### 1. Missing Value Imputation

In [36]:
def impute_missing_ae(df):
    df = df.copy()
    df['AETERM'] = df['AETERM'].fillna('Not Specified')
    df['AEDECOD'] = df['AEDECOD'].fillna('Not Specified')
    # Forward-fill dates by USUBJID (helps sequential events)
    df['AESTDTC'] = df.groupby('USUBJID')['AESTDTC'].ffill()
    df['AEENDTC'] = df.groupby('USUBJID')['AEENDTC'].ffill()
    # Impute day numbers (AESTDY/AEENDY) with per-group median if possible, else overall median to avoid empty slice warning
    for day_col in ['AESTDY', 'AEENDY']:
        if day_col in df.columns:
            overall_median = df[day_col].median()
            # Group median with fallback
            group_medians = df.groupby('USUBJID')[day_col].median()
            group_medians = group_medians.fillna(overall_median)
            df[day_col] = df[day_col].fillna(df['USUBJID'].map(group_medians))
            # Final fill if still NaN (rare)
            df[day_col] = df[day_col].fillna(overall_median)
    df = df.dropna(subset=['USUBJID'])
    print(f"Imputed days—missings left: AESTDY={df['AESTDY'].isna().sum() if 'AESTDY' in df else 'N/A'}, AEENDY={df['AEENDY'].isna().sum() if 'AEENDY' in df else 'N/A'}")
    return df

impute_missing_ae_df = impute_missing_ae(ae_df)
impute_missing_ae_df.head(5)

Imputed days—missings left: AESTDY=0, AEENDY=0


,STUDYID,DOMAIN,USUBJID,AESEQ,AESPID,AETERM,AELLT,AELLTCD,AEDECOD,AEPTCD,...,AESDISAB,AESDTH,AESHOSP,AESLIFE,AESOD,AEDTC,AESTDTC,AEENDTC,AESTDY,AEENDY
0,CDISCPILOT01,AE,01-701-1015,1.0,E07,APPLICATION SITE ERYTHEMA,APPLICATION SITE REDNESS,NaN,APPLICATION SITE ERYTHEMA,NaN,...,N,N,N,N,N,2014-01-16,2014-01-03,,2.0,10.0
1,CDISCPILOT01,AE,01-701-1015,2.0,E08,APPLICATION SITE PRURITUS,APPLICATION SITE ITCHING,NaN,APPLICATION SITE PRURITUS,NaN,...,N,N,N,N,N,2014-01-16,2014-01-03,,2.0,10.0
2,CDISCPILOT01,AE,01-701-1015,3.0,E06,DIARRHOEA,DIARRHEA,NaN,DIARRHOEA,NaN,...,N,N,N,N,N,2014-01-16,2014-01-09,2014-01-11,8.0,10.0
3,CDISCPILOT01,AE,01-701-1023,3.0,E10,ATRIOVENTRICULAR BLOCK SECOND DEGREE,AV BLOCK SECOND DEGREE,NaN,ATRIOVENTRICULAR BLOCK SECOND DEGREE,NaN,...,N,N,N,N,N,2012-08-27,2012-08-26,,22.0,26.0
4,CDISCPILOT01,AE,01-701-1023,1.0,E08,ERYTHEMA,ERYTHEMA,NaN,ERYTHEMA,NaN,...,N,N,N,N,N,2012-08-27,2012-08-07,2012-08-30,3.0,26.0


### 2. Date Standardization

In [39]:
def standardize_dates_ae(df):
    df = df.copy()
    # Parse all date cols (infer format for flexibility)
    date_cols = ['AEDTC', 'AESTDTC', 'AEENDTC']
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
    # Derived duration: Prefer date diff, fallback to rounded day diff
    date_diff = (df['AEENDTC'] - df['AESTDTC']).dt.days
    day_diff = (pd.to_numeric(df['AEENDY'], errors='coerce') - pd.to_numeric(df['AESTDY'], errors='coerce')).round()
    df['EVENT_DURATION'] = date_diff.fillna(day_diff)
    df['EVENT_DURATION'] = df['EVENT_DURATION'].fillna(0).astype('Int64')  # Safe cast after round
    print(f"Parsed dates—NaT counts: AEDTC={df['AEDTC'].isna().sum() if 'AEDTC' in df else 'N/A'}, AESTDTC={df['AESTDTC'].isna().sum()}, AEENDTC={df['AEENDTC'].isna().sum()}")
    print(f"Duration stats: mean={df['EVENT_DURATION'].mean():.1f}, min={df['EVENT_DURATION'].min()}")
    return df

standardize_dates_ae_df = standardize_dates_ae(impute_missing_ae_df)
standardize_dates_ae_df.head(5)
# standardize_dates_ae_df.columns

Parsed dates—NaT counts: AEDTC=0, AESTDTC=26, AEENDTC=473
Duration stats: mean=17.8, min=-284


,STUDYID,DOMAIN,USUBJID,AESEQ,AESPID,AETERM,AELLT,AELLTCD,AEDECOD,AEPTCD,...,AESDTH,AESHOSP,AESLIFE,AESOD,AEDTC,AESTDTC,AEENDTC,AESTDY,AEENDY,EVENT_DURATION
0,CDISCPILOT01,AE,01-701-1015,1.0,E07,APPLICATION SITE ERYTHEMA,APPLICATION SITE REDNESS,NaN,APPLICATION SITE ERYTHEMA,NaN,...,N,N,N,N,2014-01-16,2014-01-03,NaT,2.0,10.0,8
1,CDISCPILOT01,AE,01-701-1015,2.0,E08,APPLICATION SITE PRURITUS,APPLICATION SITE ITCHING,NaN,APPLICATION SITE PRURITUS,NaN,...,N,N,N,N,2014-01-16,2014-01-03,NaT,2.0,10.0,8
2,CDISCPILOT01,AE,01-701-1015,3.0,E06,DIARRHOEA,DIARRHEA,NaN,DIARRHOEA,NaN,...,N,N,N,N,2014-01-16,2014-01-09,2014-01-11,8.0,10.0,2
3,CDISCPILOT01,AE,01-701-1023,3.0,E10,ATRIOVENTRICULAR BLOCK SECOND DEGREE,AV BLOCK SECOND DEGREE,NaN,ATRIOVENTRICULAR BLOCK SECOND DEGREE,NaN,...,N,N,N,N,2012-08-27,2012-08-26,NaT,22.0,26.0,4
4,CDISCPILOT01,AE,01-701-1023,1.0,E08,ERYTHEMA,ERYTHEMA,NaN,ERYTHEMA,NaN,...,N,N,N,N,2012-08-27,2012-08-07,2012-08-30,3.0,26.0,23


### 3. Data Type Enforcement

In [42]:
def enforce_data_types_ae(df):
    df = df.copy()
    # Day cols: Numeric -> round to int -> Int64 (handles floats/NaNs)
    for day_col in ['AESTDY', 'AEENDY']:
        if day_col in df.columns:
            df[day_col] = pd.to_numeric(df[day_col], errors='coerce').round(0).astype('Int64')
    # AESEV mapping to category (title case)
    if 'AESEV' in df.columns:
        severity_map = {'MILD': 'Mild', 'MODERATE': 'Moderate', 'SEVERE': 'Severe'}
        df['AESEV'] = df['AESEV'].str.upper().map(severity_map).fillna('Unknown')
        df['AESEV'] = df['AESEV'].astype('category')
    df['USUBJID'] = df['USUBJID'].astype(str)
    if 'AESER' in df.columns:
        df['AESER'] = df['AESER'].astype('category')  # Y/N
    print("Data types enforced—no errors expected.")
    print(f"Day cols post-fix: AESTDY dtype={df['AESTDY'].dtype if 'AESTDY' in df else 'N/A'}, sample={df['AESTDY'].head() if 'AESTDY' in df else 'N/A'}")
    return df

enforce_data_types_ae_df = enforce_data_types_ae(standardize_dates_ae_df)
enforce_data_types_ae_df.head(5)

Data types enforced—no errors expected.
Day cols post-fix: AESTDY dtype=Int64, sample=0     2
1     2
2     8
3    22
4     3
Name: AESTDY, dtype: Int64


,STUDYID,DOMAIN,USUBJID,AESEQ,AESPID,AETERM,AELLT,AELLTCD,AEDECOD,AEPTCD,...,AESDTH,AESHOSP,AESLIFE,AESOD,AEDTC,AESTDTC,AEENDTC,AESTDY,AEENDY,EVENT_DURATION
0,CDISCPILOT01,AE,01-701-1015,1.0,E07,APPLICATION SITE ERYTHEMA,APPLICATION SITE REDNESS,NaN,APPLICATION SITE ERYTHEMA,NaN,...,N,N,N,N,2014-01-16,2014-01-03,NaT,2,10,8
1,CDISCPILOT01,AE,01-701-1015,2.0,E08,APPLICATION SITE PRURITUS,APPLICATION SITE ITCHING,NaN,APPLICATION SITE PRURITUS,NaN,...,N,N,N,N,2014-01-16,2014-01-03,NaT,2,10,8
2,CDISCPILOT01,AE,01-701-1015,3.0,E06,DIARRHOEA,DIARRHEA,NaN,DIARRHOEA,NaN,...,N,N,N,N,2014-01-16,2014-01-09,2014-01-11,8,10,2
3,CDISCPILOT01,AE,01-701-1023,3.0,E10,ATRIOVENTRICULAR BLOCK SECOND DEGREE,AV BLOCK SECOND DEGREE,NaN,ATRIOVENTRICULAR BLOCK SECOND DEGREE,NaN,...,N,N,N,N,2012-08-27,2012-08-26,NaT,22,26,4
4,CDISCPILOT01,AE,01-701-1023,1.0,E08,ERYTHEMA,ERYTHEMA,NaN,ERYTHEMA,NaN,...,N,N,N,N,2012-08-27,2012-08-07,2012-08-30,3,26,23


### 4. Derived Variables

In [44]:
def derive_variables_ae(df):
    df = df.copy()
    # High-Grade proxy: 1 if SEVERE (since no AETOXGR)
    if 'AESEV' in df.columns:
        df['HIGH_GRADE'] = (df['AESEV'] == 'Severe').astype(int)
    else:
        df['HIGH_GRADE'] = 0
    # Events per subject
    df['EVENT_COUNT_PER_SUBJ'] = df.groupby('USUBJID')['USUBJID'].transform('size')
    # Serious flag from AESER
    if 'AESER' in df.columns:
        df['SERIOUS_EVENT'] = (df['AESER'] == 'Y').astype(int)
    else:
        df['SERIOUS_EVENT'] = 0
    print(f"Derived stats: HIGH_GRADE=1 count={df['HIGH_GRADE'].sum()}, SERIOUS_EVENT=1 count={df['SERIOUS_EVENT'].sum()}")
    return df

derive_variables_ae_df = derive_variables_ae(enforce_data_types_ae_df)
derive_variables_ae_df.head(5)

Derived stats: HIGH_GRADE=1 count=43, SERIOUS_EVENT=1 count=3


,STUDYID,DOMAIN,USUBJID,AESEQ,AESPID,AETERM,AELLT,AELLTCD,AEDECOD,AEPTCD,...,AESOD,AEDTC,AESTDTC,AEENDTC,AESTDY,AEENDY,EVENT_DURATION,HIGH_GRADE,EVENT_COUNT_PER_SUBJ,SERIOUS_EVENT
0,CDISCPILOT01,AE,01-701-1015,1.0,E07,APPLICATION SITE ERYTHEMA,APPLICATION SITE REDNESS,NaN,APPLICATION SITE ERYTHEMA,NaN,...,N,2014-01-16,2014-01-03,NaT,2,10,8,0,3,0
1,CDISCPILOT01,AE,01-701-1015,2.0,E08,APPLICATION SITE PRURITUS,APPLICATION SITE ITCHING,NaN,APPLICATION SITE PRURITUS,NaN,...,N,2014-01-16,2014-01-03,NaT,2,10,8,0,3,0
2,CDISCPILOT01,AE,01-701-1015,3.0,E06,DIARRHOEA,DIARRHEA,NaN,DIARRHOEA,NaN,...,N,2014-01-16,2014-01-09,2014-01-11,8,10,2,0,3,0
3,CDISCPILOT01,AE,01-701-1023,3.0,E10,ATRIOVENTRICULAR BLOCK SECOND DEGREE,AV BLOCK SECOND DEGREE,NaN,ATRIOVENTRICULAR BLOCK SECOND DEGREE,NaN,...,N,2012-08-27,2012-08-26,NaT,22,26,4,0,4,0
4,CDISCPILOT01,AE,01-701-1023,1.0,E08,ERYTHEMA,ERYTHEMA,NaN,ERYTHEMA,NaN,...,N,2012-08-27,2012-08-07,2012-08-30,3,26,23,0,4,0


### 5. Outlier Detection/Cleaning

In [45]:
def clean_outliers_ae(df):
    df = df.copy()
    print("Before filters:", df.shape)
    # Remove negative durations
    df = df[df['EVENT_DURATION'] >= 0]
    print("After duration filter:", df.shape)
    # Dedup by USUBJID + AESTDTC
    df = df.drop_duplicates(subset=['USUBJID', 'AESTDTC'])
    print("After duplicates:", df.shape)
    return df

clean_outliers_ae_df = clean_outliers_ae(derive_variables_ae_df)
clean_outliers_ae_df.head(5)

Before filters: (1191, 39)
After duration filter: (1012, 39)
After duplicates: (553, 39)


,STUDYID,DOMAIN,USUBJID,AESEQ,AESPID,AETERM,AELLT,AELLTCD,AEDECOD,AEPTCD,...,AESOD,AEDTC,AESTDTC,AEENDTC,AESTDY,AEENDY,EVENT_DURATION,HIGH_GRADE,EVENT_COUNT_PER_SUBJ,SERIOUS_EVENT
0,CDISCPILOT01,AE,01-701-1015,1.0,E07,APPLICATION SITE ERYTHEMA,APPLICATION SITE REDNESS,NaN,APPLICATION SITE ERYTHEMA,NaN,...,N,2014-01-16,2014-01-03,NaT,2,10,8,0,3,0
2,CDISCPILOT01,AE,01-701-1015,3.0,E06,DIARRHOEA,DIARRHEA,NaN,DIARRHOEA,NaN,...,N,2014-01-16,2014-01-09,2014-01-11,8,10,2,0,3,0
3,CDISCPILOT01,AE,01-701-1023,3.0,E10,ATRIOVENTRICULAR BLOCK SECOND DEGREE,AV BLOCK SECOND DEGREE,NaN,ATRIOVENTRICULAR BLOCK SECOND DEGREE,NaN,...,N,2012-08-27,2012-08-26,NaT,22,26,4,0,4,0
4,CDISCPILOT01,AE,01-701-1023,1.0,E08,ERYTHEMA,ERYTHEMA,NaN,ERYTHEMA,NaN,...,N,2012-08-27,2012-08-07,2012-08-30,3,26,23,0,4,0
7,CDISCPILOT01,AE,01-701-1028,1.0,E04,APPLICATION SITE ERYTHEMA,APPLICATION SITE ERYTHEMA,NaN,APPLICATION SITE ERYTHEMA,NaN,...,N,2013-08-01,2013-07-21,NaT,3,53,50,0,2,0


### 6. Consistency Checks

In [46]:
def consistency_checks_ae(df):
    df = df.copy()
    # Basic MedDRA validation (substring for demo; keep if contains 'MedDRA' or as-is)
    if 'AEDECOD' in df.columns:
        df['AEDECOD'] = df['AEDECOD'].where(df['AEDECOD'].str.contains('MedDRA', na=False, case=False), df['AEDECOD'])
    # Ensure AEBODSYS non-null
    if 'AEBODSYS' in df.columns:
        df = df.dropna(subset=['AEBODSYS'])
    return df

consistency_checks_ae_df = consistency_checks_ae(clean_outliers_ae_df)
consistency_checks_ae_df.head(5)

,STUDYID,DOMAIN,USUBJID,AESEQ,AESPID,AETERM,AELLT,AELLTCD,AEDECOD,AEPTCD,...,AESOD,AEDTC,AESTDTC,AEENDTC,AESTDY,AEENDY,EVENT_DURATION,HIGH_GRADE,EVENT_COUNT_PER_SUBJ,SERIOUS_EVENT
0,CDISCPILOT01,AE,01-701-1015,1.0,E07,APPLICATION SITE ERYTHEMA,APPLICATION SITE REDNESS,NaN,APPLICATION SITE ERYTHEMA,NaN,...,N,2014-01-16,2014-01-03,NaT,2,10,8,0,3,0
2,CDISCPILOT01,AE,01-701-1015,3.0,E06,DIARRHOEA,DIARRHEA,NaN,DIARRHOEA,NaN,...,N,2014-01-16,2014-01-09,2014-01-11,8,10,2,0,3,0
3,CDISCPILOT01,AE,01-701-1023,3.0,E10,ATRIOVENTRICULAR BLOCK SECOND DEGREE,AV BLOCK SECOND DEGREE,NaN,ATRIOVENTRICULAR BLOCK SECOND DEGREE,NaN,...,N,2012-08-27,2012-08-26,NaT,22,26,4,0,4,0
4,CDISCPILOT01,AE,01-701-1023,1.0,E08,ERYTHEMA,ERYTHEMA,NaN,ERYTHEMA,NaN,...,N,2012-08-27,2012-08-07,2012-08-30,3,26,23,0,4,0
7,CDISCPILOT01,AE,01-701-1028,1.0,E04,APPLICATION SITE ERYTHEMA,APPLICATION SITE ERYTHEMA,NaN,APPLICATION SITE ERYTHEMA,NaN,...,N,2013-08-01,2013-07-21,NaT,3,53,50,0,2,0


### 7. Export Prep

In [47]:
def prepare_export_ae(df):
    df = df.copy()
    col_order = ['USUBJID', 'AESTDTC', 'AEENDTC', 'AETERM', 'AEDECOD', 'AETOXGR', 'HIGH_GRADE', 'EVENT_DURATION', 'SERIOUS_EVENT', 'EVENT_COUNT_PER_SUBJ']
    existing_cols = [col for col in col_order if col in df.columns]
    remaining_cols = [col for col in df.columns if col not in col_order]
    df = df.reindex(columns=existing_cols + remaining_cols)
    df['ETL_TIMESTAMP'] = pd.Timestamp.now()
    # Oncology risk flag
    if 'AETERM' in df.columns:
        df['ONCOLOGY_RISK'] = df['AETERM'].str.contains('neoplasm|cancer|tumor', case=False, na=False).astype(int)
    return df

prepare_export_ae_df = prepare_export_ae(consistency_checks_ae_df)
prepare_export_ae_df.head(5)

,USUBJID,AESTDTC,AEENDTC,AETERM,AEDECOD,HIGH_GRADE,EVENT_DURATION,SERIOUS_EVENT,EVENT_COUNT_PER_SUBJ,STUDYID,...,AESDISAB,AESDTH,AESHOSP,AESLIFE,AESOD,AEDTC,AESTDY,AEENDY,ETL_TIMESTAMP,ONCOLOGY_RISK
0,01-701-1015,2014-01-03,NaT,APPLICATION SITE ERYTHEMA,APPLICATION SITE ERYTHEMA,0,8,0,3,CDISCPILOT01,...,N,N,N,N,N,2014-01-16,2,10,2025-09-25 00:57:23.624132,0
2,01-701-1015,2014-01-09,2014-01-11,DIARRHOEA,DIARRHOEA,0,2,0,3,CDISCPILOT01,...,N,N,N,N,N,2014-01-16,8,10,2025-09-25 00:57:23.624132,0
3,01-701-1023,2012-08-26,NaT,ATRIOVENTRICULAR BLOCK SECOND DEGREE,ATRIOVENTRICULAR BLOCK SECOND DEGREE,0,4,0,4,CDISCPILOT01,...,N,N,N,N,N,2012-08-27,22,26,2025-09-25 00:57:23.624132,0
4,01-701-1023,2012-08-07,2012-08-30,ERYTHEMA,ERYTHEMA,0,23,0,4,CDISCPILOT01,...,N,N,N,N,N,2012-08-27,3,26,2025-09-25 00:57:23.624132,0
7,01-701-1028,2013-07-21,NaT,APPLICATION SITE ERYTHEMA,APPLICATION SITE ERYTHEMA,0,50,0,2,CDISCPILOT01,...,N,N,N,N,N,2013-08-01,3,53,2025-09-25 00:57:23.624132,0
